In [1]:
import pandas as pd
import pywhatkit as wpp
from datetime import datetime 
import time

#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',False)

In [2]:
path = 'Data/'
dia = datetime.now().day
sem = (int(datetime.now().strftime('%U'))) //4 - 1
mes = datetime.now().month
year = datetime.now().year
#print("BASE BLENDED NL S{:d}M{:02d} {}.xlsx".format(sem, mes, year))
#print('Román - M{:d} Seguimiento.xlsx'.format(mes))
#print('Alertas tempranas {:2} de Abril 2024.xlsx'.format(dia))
df_matries = pd.read_excel(path + 'Román - M{:d} Seguimiento.xlsx'.format(mes), sheet_name='Asignación')
database = pd.read_excel(path + "BASE BLENDED NL S{:d}M{:02d} {}.xlsx".format(sem, mes, year), sheet_name = 'ACTIVOS')
#inactivos = pd.read_excel(path + 'Alertas tempranas {:2} de Abril 2024.xlsx'.format(dia), sheet_name='Inactivo')

In [3]:
#Se eliminan columnas hasta la columna Teléfono 2.
columna_limite = "Teléfono2"
columnas_conservadas = database.columns[:database.columns.get_loc(columna_limite) + 1]
database = database[columnas_conservadas] 

#Se elimina USUARIO BB,PASSWORD PORTAL ALUMNO, PROYECTO, MODELO EDUCATIVO, SERVICIO, Modalidad, Monto, Correo 2
database.drop(columns = ["USUARIO BB","PASSWORD PORTAL \nALUMNO", 'PROYECTO','MODELO EDUCATIVO', 'Fecha', 'SERVICIO', 'Fecha de inicio','Modalidad', 'Monto', 'Correo 2'], inplace=True)

#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE"}, inplace= True)
database.rename(columns = {'MATRICULA' : "Matricula"}, inplace= True)
#Cambia el formato de las columnas TELÉFONO y CELULAR
#database_final["TELÉFONO"] = database_final["TELÉFONO"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
#database_final["CELULAR"] = database_final["CELULAR"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']
database

,NOMBRE,Matricula,CE CANALIZADO,CARRERA,M4,Correo,Teléfono,Teléfono2
1,LAURA OFELIA SOTO CELIS,BNL032757,CE CADEREYTA,CP,2,laurasotocelis89@gmail.com,8282813403,8135710251
8,MARIA CAROLINA PEREZ RAMOS,UVNL020298,CE CADEREYTA,LAE,3,carolinaramosso06@gmail.com,8132681273,NaN
10,ADRIANA GARZA ALARDIN,BNL049990,CE CADEREYTA,LCE,3,adriana_03garza@hotmail.com,8281190978,NaN
32,JESUS TADEO ACEVEDO ACEVEDO,BNL059688,CE CADEREYTA,IIS,3,jt.acevedox2@gmail.com,8281001361,NaN
81,JORGE ARMANDO VALDES SANCHEZ,BNL065348,CE CADEREYTA,LAE,1,jorgevaldes0329@gmail.com,8135661981,NaN
...,...,...,...,...,...,...,...,...
2718,JONATHAN ISRAEL SOTO CORTEZ,BNL104166,CE CADEREYTA,IIS,Inducción M4 2024,jonathanisraelsotocortez7@gmail.com,8180110941,NaN
2721,LORENZO ANTONIO BOTELLO MARTINEZ,BNL104990,CE CADEREYTA,IGE,Inducción M4 2024,lastmasmi23@gmail.com,8128669065,NaN
2740,ULISES CALTE AGUILAR,BNL104543,CE CADEREYTA,IIS,Inducción M4 2024,calteaguilaru@gmail.com,2281967311,8123618439
2768,LUIS ELIUD SALINAS SOTO,BNL105089,CE CADEREYTA,LAE,Inducción M4 2024,luisless22@gmail.com,8127446548,8116108199


### Para eliminar columnas del de ALERTAS TEMPRANAS 

In [4]:
#eliminar_columnas = [col for col in df_inactivos.columns if col.startswith('Registro')]
#df_inactivos.drop(columns = eliminar_columnas, inplace=True)
#df_inactivos

### Filtros a los Inactivos.

In [5]:
df_matries = df_matries[df_matries['CE'] == 'CE CADEREYTA']
df_matries.shape

(320, 10)

##### Para Alumnos de (tal materia)

In [6]:
link = 'https://chat.whatsapp.com/Env3dGXvLIl7HYXWFUtICE'
imagen = 'img/FunMat.png'

df_matries = df_matries[df_matries['MATERIA'] == 'Fundamentos de matemáticas']

Alumno de baja lo eliminamos

In [7]:
#df_matries = df_matries.drop(df_matries[df_matries['MATRICULA'] == 'bnl103337'].index)
matric_num_equiv = ['bnl100961','bnl103801', 'bnl103876']

### Ciclo de Enviado

In [8]:
for index, row in df_matries.iterrows():
    alumno = row['NOMBRE']
    matricula = str(row['MATRICULA'].upper())
    materia = row['MATERIA']
    telefono = database.loc[database['Matricula'] == matricula, 'Teléfono'].values
    telefono2 = database.loc[database['Matricula'] == matricula, 'Teléfono2'].values
    
    if len(telefono) > 0:
        if matricula in (matric_num_equiv):
            telefono = telefono2
        telefono = '+52 ' + str(int(telefono[0]))        
    
    mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte al grupo colaborativo de la materia: *'+ materia + '* con la finalidad de que podamos entablar conversación por ahí sobre los temas relacionados y además tener apoyo entre los compañeros. \n\nSi gustas a continuación te dejo el enlace: ' + link
    hora = datetime.now().strftime('%H:%M')

    #print(matricula,'17/04',' 16:00 Alumno se une al grupo colaborativo de Fundamentos de Matemáticas.')
    #print(matricula,'17/04',' 16:00 Alumno se le notifica sobre el grupo colaborativo de Fundamentos de Matemáticas.')
    #print('\n')
    #print(telefono)
    #wpp.sendwhats_image(telefono, imagen, mensaje, 20, True)
    #time.sleep(60)